In [2]:
import logging
import math
import scipy as sp
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torchmetrics.regression import MeanAbsolutePercentageError
import pennylane as qml

import matplotlib.pyplot as plt
plt.style.use('./pptnqfe.mplstyle')

from qulearn.hat_basis import HatBasis
from qulearn.trainer import SupervisedTrainer
from qulearn.qlayer import (HatBasisQFE,
                            TwoQubitRotCXMPSLayer,
                            embedU,
                            ParallelIQPEncoding,
                            AltRotCXLayer,
                            MeasurementLayer,
                            HamiltonianLayer,
                            MeasurementType)

# Section 5.2

In [ ]:
num_qubits = 2
num_nodes = 2**num_qubits
a = -1.0
b = 1.0
hat_basis = HatBasis(a=a, b=b, num_nodes=num_nodes)

num_pnts = 500
xvals = torch.linspace(-1.0, 1.0, num_pnts)
basis_vectors = hat_basis.eval_basis_vector(xvals)
basis_vectors = torch.sqrt(basis_vectors)

num_subplots = num_nodes * (num_nodes + 1) // 2
fig, axs = plt.subplots(3, 4, figsize=(15, 10))
axs_flat = axs.flatten()

subplot_idx = 0
for i in range(num_nodes):
    for j in range(i + 1):
        axs_flat[subplot_idx].plot(xvals, basis_vectors[:, i] * basis_vectors[:, j])
        axs_flat[subplot_idx].set_title(f'$\\varphi_{i}\\varphi_{j}$')
        axs_flat[subplot_idx].set_xlabel('$x$')
        subplot_idx += 1

for idx in range(subplot_idx, len(axs_flat)):
    fig.delaxes(axs_flat[idx])

plt.tight_layout()
plt.savefig('./figures/basis_funcs.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [4]:
num_qubits = 5
num_nodes = 2**num_qubits
a = -1.0
b = 1.0
hat_basis = HatBasis(a=a, b=b, num_nodes=num_nodes)

embed = HatBasisQFE(wires=num_qubits, basis=hat_basis, sqrt=True, normalize=False)
obs = qml.PauliZ(0)
model = MeasurementLayer(embed, observables=obs, measurement_type=MeasurementType.Expectation)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([0.0])
print(drawer(x))

0: ──────────────────────╭U(M2)─┤  <Z>
1: ───────────────╭U(M1)─╰U(M2)─┤     
2: ────────╭U(M1)─╰U(M1)────────┤     
3: ─╭U(M0)─╰U(M1)───────────────┤     
4: ─╰U(M0)──────────────────────┤     


In [ ]:
num_pnts = 500
xvals = torch.linspace(-1.0, 1.0, num_pnts).unsqueeze(-1)
yvals = model(xvals)

plt.figure(figsize=(10, 6))
plt.plot(xvals, yvals)
plt.xlabel('$x$')
plt.title("$\langle Z_0\\rangle$")
plt.savefig('./figures/z0.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
obs = qml.PauliZ(num_qubits-1)
model = MeasurementLayer(embed, observables=obs, measurement_type=MeasurementType.Expectation)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([0.0])
print(drawer(x))

In [ ]:
yvals = model(xvals)
plt.figure(figsize=(10, 6))
plt.plot(xvals, yvals)
plt.xlabel('$x$')
plt.title("$\langle Z_4\\rangle$")
plt.savefig('./figures/zn.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
obs = qml.PauliX(4)
model = MeasurementLayer(embed, observables=obs, measurement_type=MeasurementType.Expectation)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([0.0])
print(drawer(x))

In [ ]:
yvals = model(xvals)
plt.figure(figsize=(10, 6))
plt.plot(xvals, yvals)
plt.xlabel('$x$')
plt.title("$\langle X_4\\rangle$")
plt.savefig('./figures/xn.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
def step_function(
    X, threshold1=-1.0, threshold2=0.0, low_value=0.0, mid_value=2, high_value=-1.0
):
    condition1 = X < threshold1
    condition2 = (X >= threshold1) & (X < threshold2)
    condition3 = X >= threshold2

    values = torch.zeros_like(X)
    values[condition1] = low_value
    values[condition2] = mid_value
    values[condition3] = high_value

    return values

def add_gaussian_noise(tensor, mean=0.0, std=0.01):
    return tensor + torch.randn(tensor.size()) * std + mean

X = torch.linspace(-1, 1, 1000).reshape(-1, 1)
func = step_function
sigma = 0.1
Y = add_gaussian_noise(func(X), std=sigma)
plt.plot(X, Y)
plt.savefig('./figures/stepfunc.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
obs = [qml.Identity(0), qml.PauliZ(0)]
model = HamiltonianLayer(embed, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([0.0])
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

In [ ]:
N_train = 50
N_valid = 10
batch_size = 10
X_train = torch.linspace(-1, 1, N_train, dtype=torch.float64).reshape(-1, 1)
Y_train = add_gaussian_noise(func(X_train), std=sigma)
X_valid = torch.linspace(-1, 1, N_valid, dtype=torch.float64).reshape(-1, 1)
Y_valid = func(X_valid)
data_train = TensorDataset(X_train, Y_train)
data_valid = TensorDataset(X_valid, Y_valid)
loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(data_valid, batch_size=batch_size, shuffle=True)

lr = 0.1
optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)
loss_fn = torch.nn.MSELoss()
metric = MeanAbsolutePercentageError()

logger = logging.getLogger("train_function")
logger.setLevel(level=logging.INFO)
num_epochs = 100
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/stepfunc_train_pptnqfe.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
num_features = 1
base = 3.0
omega = 1.0
embed = ParallelIQPEncoding(wires=num_qubits,
                            num_features=num_features,
                            n_repeat=1,
                            base=base,
                            omega=omega)
n_layers = 3
var = AltRotCXLayer(wires=num_qubits, n_layers=n_layers)

obs = [qml.Identity(0), qml.PauliZ(0)]
model = HamiltonianLayer(embed, var, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([1.0])
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/stepfunc_train_iqpe.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
def trigonometric(X, a=1, b=math.pi, c=0.):
    return a * torch.sin(b * X + c)

X = torch.linspace(-1, 1, 1000).reshape(-1, 1)
func = trigonometric
sigma = 0.1
Y = add_gaussian_noise(func(X), std=sigma)
plt.plot(X, Y)
plt.savefig('./figures/sin.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [5]:
embed = HatBasisQFE(wires=num_qubits, basis=hat_basis, sqrt=True, normalize=False)
num_mps_layers = 1
num_block_layers = 3
reverse = True
var = TwoQubitRotCXMPSLayer(num_qubits, 
                            n_layers_mps=num_mps_layers,
                            n_layers_block=num_block_layers,
                            reverse=reverse)

obs = [qml.Identity(0)] + [qml.PauliZ(j) for j in range(num_qubits)]
model = HamiltonianLayer(embed, var, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([0.0])
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

0: ──────────────────────────────────────────────────╭U(M2)────────────────Rot(1.36,5.74,5.80)───
1: ─────────────────────────────╭U(M1)───────────────╰U(M2)────────────────Rot(1.14,4.92,2.64)───
2: ────────╭U(M1)───────────────╰U(M1)────────────────Rot(3.20,4.89,0.33)────────────────────────
3: ─╭U(M0)─╰U(M1)────────────────Rot(3.94,5.34,3.90)─╭●────────────────────Rot(3.33,1.74,4.82)─╭●
4: ─╰U(M0)──Rot(5.16,1.06,1.27)──────────────────────╰X────────────────────Rot(5.56,4.58,1.01)─╰X

─────────────────────────────────────────────────────────────────────────────────────────────────
─────────────────────────────────────────────────────────────────────────────────────────────────
───────────────────────────────────────────────╭●──Rot(1.65,5.91,2.03)─╭●──Rot(0.29,3.72,2.42)─╭●
───Rot(0.32,3.15,1.61)─╭●──Rot(0.54,3.91,0.48)─╰X──Rot(0.60,0.95,4.78)─╰X──Rot(1.97,0.02,2.60)─╰X
───Rot(4.37,3.11,5.60)─╰X──Rot(2.57,3.67,2.41)───────────────────────────────────────────────────

──────────────────

In [ ]:
N_train = 50
N_valid = 10
batch_size = 10
X_train = torch.linspace(-1, 1, N_train, dtype=torch.float64).reshape(-1, 1)
Y_train = add_gaussian_noise(func(X_train), std=sigma)
X_valid = torch.linspace(-1, 1, N_valid, dtype=torch.float64).reshape(-1, 1)
Y_valid = func(X_valid)
data_train = TensorDataset(X_train, Y_train)
data_valid = TensorDataset(X_valid, Y_valid)
loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(data_valid, batch_size=batch_size, shuffle=True)

optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)
num_epochs = 100
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/sin_train_pptnqfe.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
num_features = 1
base = 3.0
omega = 1.0
embed = ParallelIQPEncoding(wires=num_qubits,
                            num_features=num_features,
                            n_repeat=1,
                            base=base,
                            omega=omega)
n_layers = 3
var = AltRotCXLayer(wires=num_qubits, n_layers=n_layers)

obs = [qml.Identity(0)] + [qml.PauliZ(j) for j in range(num_qubits)]
model = HamiltonianLayer(embed, var, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([1.0])
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/sin_train_iqpe.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
n_layers = 1
var = AltRotCXLayer(wires=num_qubits, n_layers=n_layers)

obs = [qml.Identity(0)] + [qml.PauliZ(j) for j in range(num_qubits)]
model = HamiltonianLayer(embed, var, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([1.0])
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

In [ ]:
optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/sin_train_iqpe_l0.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()

In [6]:
num_qubits = 5
def construct_matrix(c):
    size = 2**num_qubits
    H = np.zeros((size, size), dtype=complex)
    np.fill_diagonal(H[:2], 1)
    np.fill_diagonal(H[-2:], -1)

    counter = 0
    for i in range(size):
        for j in range(i+1, size):
            H[i, j] = complex(0, c[counter])
            counter += 1

    H = H + H.conj().T - np.diag(H.diagonal())
    return H

def objective_function(c):
    H = construct_matrix(c)
    size = H.shape[0]
    half = int(size/2)
    eigenvalues = np.linalg.eigvalsh(H)
    eigenvalues_sorted = np.sort(eigenvalues)[::-1]
    target_eigenvalues = [1]*half+[-1]*half
    
    error = np.sum((eigenvalues_sorted - target_eigenvalues)**2)
    return error

num_params = (2**num_qubits * (2**num_qubits - 1)) // 2
initial_guesses = np.ones(num_params)
result = sp.optimize.minimize(objective_function, initial_guesses, method='BFGS')

optimized_c = result.x
print("Cost value:", result.fun)

H = construct_matrix(optimized_c)
eigs, U = np.linalg.eigh(H)
print(eigs)

Cost value: 1.0557280902123383
[-1.00000425 -1.00000176 -1.00000105 -1.00000063 -1.00000044 -1.00000013
 -0.99999987 -0.99999966 -0.99999941 -0.99999901 -0.9999989  -0.99999868
 -0.99999836 -0.99999774 -0.61803559 -0.61803239  0.99999774  0.99999836
  0.99999868  0.9999989   0.99999901  0.99999941  0.99999966  0.99999987
  1.00000013  1.00000044  1.00000063  1.00000105  1.00000176  1.00000425
  1.61803294  1.61803504]


In [7]:
a = -1.0
b = 1.0
num_nodes = 2**num_qubits
hat_basis = HatBasis(a=a, b=b, num_nodes=num_nodes)      
embed = HatBasisQFE(wires=num_qubits, basis=hat_basis, sqrt=True, normalize=False)
var = embedU(num_qubits, U.conj().T)

obs = [qml.PauliZ(0)]
model = HamiltonianLayer(embed, var, observables=obs)
drawer = qml.draw(model.qnode, show_all_wires=True, expansion_strategy="device")
x = torch.tensor([-0.1429])
y = model(x)
print(y)
print(drawer(x))
nump = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: ", nump)

tensor([4.5533e-14], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
0: ──────────────────────╭U(M2)─╭U(M3)─┤  <𝓗(0.65)>
1: ───────────────╭U(M1)─╰U(M2)─├U(M3)─┤           
2: ────────╭U(M1)─╰U(M1)────────├U(M3)─┤           
3: ─╭U(M0)─╰U(M1)───────────────├U(M3)─┤           
4: ─╰U(M0)──────────────────────╰U(M3)─┤           
Number of parameters:  1


In [ ]:
with torch.no_grad():
    yvals = model(xvals)
plt.figure(figsize=(10, 6))
plt.plot(xvals, yvals)
plt.xlabel('$x$')
plt.show()

In [ ]:
def step_function(
    X, threshold1=-0.9, threshold2=0.9, low_value=2.0, mid_value=0, high_value=-2.0
):
    condition1 = X < threshold1
    condition2 = (X >= threshold1) & (X < threshold2)
    condition3 = X >= threshold2

    values = torch.zeros_like(X)
    values[condition1] = low_value
    values[condition2] = mid_value
    values[condition3] = high_value

    return values

X = torch.linspace(-1, 1, 1000).reshape(-1, 1)
func = step_function
sigma = 0.1
Y = add_gaussian_noise(func(X), std=sigma)
plt.plot(X, Y)
plt.savefig('./figures/segments.pdf', format='pdf', bbox_inches='tight')
plt.show()

In [ ]:
N_train = 50
N_valid = 10
batch_size = 10
X_train = torch.linspace(-1, 1, N_train, dtype=torch.float64).reshape(-1, 1)
Y_train = add_gaussian_noise(func(X_train), std=sigma)
X_valid = torch.linspace(-1, 1, N_valid, dtype=torch.float64).reshape(-1, 1)
Y_valid = func(X_valid)
data_train = TensorDataset(X_train, Y_train)
data_valid = TensorDataset(X_valid, Y_valid)
loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(data_valid, batch_size=batch_size, shuffle=True)

lr = 0.1
optimizer = Adam(model.parameters(), lr=lr, amsgrad=True)

num_epochs = 100
trainer = SupervisedTrainer(
    optimizer=optimizer,
    loss_fn=loss_fn,
    metrics={"MARE": metric},
    num_epochs=num_epochs,
    logger=logger,
)

In [ ]:
trainer.train(model, train_data=loader_train, valid_data=loader_valid)

In [ ]:
X = torch.linspace(-1, 1, 300, dtype=torch.float64).reshape(-1, 1)
Y_exact = func(X)
model.eval()
with torch.no_grad():
    Y_model = model(X)
plt.figure(figsize=(10, 6))
plt.plot(X, Y_exact, label="exact", color="blue")
plt.plot(X, Y_model, label="predicted", color="red")
plt.xlabel("$x$")
plt.legend()
plt.savefig('./figures/twosteps_train_pptnqfe.pdf', format='pdf', bbox_inches='tight')
plt.tight_layout()
plt.show()